In [1]:
import pandas as pd
data = pd.read_csv("./aos_data/merged_phuc_data.csv")  # Assuming you save your data as CSV


In [2]:

from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Example: Normalize raw features
scaler = MinMaxScaler()
features = ['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ', 'IR']
data[features] = scaler.fit_transform(data[features])

# Windowing (simplified, assumes 5 samples per window)
window_size = 5
X, y = [], []
for i in range(0, len(data) - window_size, window_size):
    window = data.iloc[i:i+window_size][features].mean().values  # Mean of window
    label = data.iloc[i:i+window_size]['Label'].mode()[0]  # Majority label
    X.append(window)
    y.append(1 if label == 'awake' else 0)
X, y = np.array(X), np.array(y)


In [3]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(len(features),)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
233/233 [==============================] - 2s 4ms/step - loss: 0.4901 - accuracy: 0.9140 - val_loss: 0.2735 - val_accuracy: 0.9973
Epoch 2/20
233/233 [==============================] - 0s 2ms/step - loss: 0.1313 - accuracy: 0.9973 - val_loss: 0.0501 - val_accuracy: 0.9973
Epoch 3/20
233/233 [==============================] - 0s 2ms/step - loss: 0.0305 - accuracy: 0.9976 - val_loss: 0.0179 - val_accuracy: 0.9978
Epoch 4/20
233/233 [==============================] - 0s 2ms/step - loss: 0.0133 - accuracy: 0.9987 - val_loss: 0.0093 - val_accuracy: 0.9989
Epoch 5/20
233/233 [==============================] - 0s 2ms/step - loss: 0.0074 - accuracy: 0.9996 - val_loss: 0.0055 - val_accuracy: 0.9995
Epoch 6/20
233/233 [==============================] - 0s 2ms/step - loss: 0.0046 - accuracy: 0.9996 - val_loss: 0.0036 - val_accuracy: 1.0000
Epoch 7/20
233/233 [==============================] - 0s 2ms/step - loss: 0.0030 - accuracy: 0.9997 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Optimize for size
tflite_model = converter.convert()

# Save the model
with open('sleep_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\nguye\AppData\Local\Temp\tmpim_o9w0i\assets


In [5]:
# Extract min and max values
min_values = scaler.data_min_
max_values = scaler.data_max_

# Print them for reference
for feature, min_val, max_val in zip(features, min_values, max_values):
    print(f"{feature}: min = {min_val}, max = {max_val}")

AccelX: min = 9.375694, max = 10.74997
AccelY: min = -0.88346, max = 2.142811
AccelZ: min = -1.563414, max = 2.619257
GyroX: min = -1.031476, max = 0.501216
GyroY: min = -1.821804, max = 1.174034
GyroZ: min = -0.252606, max = 0.099657
IR: min = 974.0, max = 133423.0
